Growthlist Scraper 

> Scraper

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    startups_data = []
    table_rows = soup.select('tr[data-row_id]')

    for row in table_rows:
        columns = row.find_all('td')
        if len(columns) > 6:  
            data = {
                'Name': columns[0].get_text(strip=True) if columns[0] else None,
                'Website': columns[1].get_text(strip=True) if columns[1] else None,
                'Industry': columns[2].get_text(strip=True) if columns[2] else None,
                'Country': columns[3].get_text(strip=True) if columns[3] else None,
                'Funding Amount (USD)': columns[4].get_text(strip=True) if columns[4] else None,
                'Funding Type': columns[5].get_text(strip=True) if columns[5] else None,
                'Last Funding Date': columns[6].get_text(strip=True) if columns[6] else None,
            }
            startups_data.append(data)

    df = pd.DataFrame(startups_data)
    return df

> List of URLs

> Execute - run on URLs and combine into one big dataframe

In [25]:
# Organizing URLs
urls = {
    'Location': {
        '': '',
    },
    'Industry': {
        '': '',
    },
    'Niche': {
        '': '',  
    }
}

# List to store each category's DataFrame
dfs = []

# Loop through the structured dictionary
for category, category_urls in urls.items():
    for key, url in category_urls.items():
        df = scrape(url)  
        df['Category'] = category  # Add column based on dictionary value
        df['Subcategory'] = key  
        dfs.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

> Filter - remove duplicates, none B2B in industry column, not in USA

In [26]:
combined_df = combined_df.drop_duplicates(subset=['Name'])
combined_df = combined_df[combined_df['Country'] == 'United States']
combined_df = combined_df.drop(columns=['Country'])
combined_df = combined_df[combined_df['Industry'].str.contains('B2B')]

combined_df['Last Funding Date'] = pd.to_datetime(combined_df['Last Funding Date'], format='%b %Y')
combined_df = combined_df.sort_values(by='Last Funding Date', ascending=False)


> Export to csv

In [27]:
# Export combined_df to a CSV file
combined_df.to_csv('Growthlist Startups.csv', index=False)